In [133]:

import pandas as pd
import numpy as np


In [134]:

df = pd.read_csv('100_Unique_QA_Dataset.csv')
df.head()


,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


# Tokenize

In [135]:

def tokenize(text):
  text = text.lower().replace('?', '').replace("'", '')
  return text.split()


In [136]:

tokenize(df.question[0])


['what', 'is', 'the', 'capital', 'of', 'france']

# **Vocab**

In [137]:

vocab = {'<UNK>': 0}
def build_vocab(row):

  tokenized_question = tokenize(row["question"])
  tokenized_answer = tokenize(row["answer"])

  final_token = tokenized_question + tokenized_answer

  for token in final_token:
    vocab[token] = len(vocab)



In [138]:

df.apply(build_vocab, axis=1)


,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [139]:
print(len(vocab))

324


# Convert words to numerical indices

In [140]:
def text_to_indices(text, vocab):

  indexed_text=[]
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])
  return indexed_text

In [141]:
text_to_indices(df.question[0], vocab)

[317, 322, 322, 317, 320, 324]

In [142]:

import torch
from torch.utils.data import Dataset, DataLoader


In [143]:
class QADataset(Dataset):

  def __init__(self, df , vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    question = self.df.iloc[idx]['question']
    answer = self.df.iloc[idx]['answer']
    question_indices = text_to_indices(question, self.vocab)
    answer_indices = text_to_indices(answer, self.vocab)

    return torch.tensor(question_indices), torch.tensor(answer_indices)


In [144]:

dataset = QADataset(df, vocab)


In [145]:
dataset[0]

(tensor([317, 322, 322, 317, 320, 324]), tensor([7]))

In [146]:

dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


In [147]:

for question , answer in dataloader:
  print(question)
  print(answer)


tensor([[322, 101, 322, 322, 168]])
tensor([[102]])
tensor([[308, 270, 322, 277, 270, 271, 271, 320, 322, 272]])
tensor([[273]])
tensor([[308, 130, 322,  30,  31]])
tensor([[32]])
tensor([[317, 322, 322, 245, 245, 320, 245]])
tensor([[246]])
tensor([[317, 322, 322,  59,  60, 320, 134, 297,  60]])
tensor([[61]])
tensor([[322, 216, 301, 217, 218, 297, 301, 219, 220]])
tensor([[220]])
tensor([[317, 322, 322, 161, 161, 322, 161]])
tensor([[162]])
tensor([[308, 189, 282,  13, 301,  15]])
tensor([[16]])
tensor([[288, 288, 129, 289, 297, 322, 130, 130]])
tensor([[263]])
tensor([[317, 322, 322, 317, 320, 135]])
tensor([[136]])
tensor([[308, 130, 130, 236]])
tensor([[132]])
tensor([[317, 322, 322, 185, 271, 120]])
tensor([[95]])
tensor([[308, 296, 111]])
tensor([[112]])
tensor([[317, 241, 229, 230, 231, 232]])
tensor([[233]])
tensor([[308, 308, 322, 309, 310]])
tensor([[311]])
tensor([[322, 281, 322, 322, 281, 282, 322, 282]])
tensor([[283]])
tensor([[322, 312, 322, 313, 322, 319, 322, 314, 315

In [148]:

import torch.nn as nn
import torch.nn.functional as F


In [176]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output,hidden

In [177]:
vocab_size = 325
learning_rate = 0.001
epochs = 30
model = SimpleRNN(vocab_size)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


In [179]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    output, hidden = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    loss.backward()

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 3.575779
Epoch: 2, Loss: 3.293026
Epoch: 3, Loss: 3.037149
Epoch: 4, Loss: 2.812011
Epoch: 5, Loss: 2.607185
Epoch: 6, Loss: 2.426938
Epoch: 7, Loss: 2.254099
Epoch: 8, Loss: 2.100435
Epoch: 9, Loss: 1.962426
Epoch: 10, Loss: 1.833710
Epoch: 11, Loss: 1.713939
Epoch: 12, Loss: 1.609448
Epoch: 13, Loss: 1.509960
Epoch: 14, Loss: 1.418099
Epoch: 15, Loss: 1.334443
Epoch: 16, Loss: 1.254332
Epoch: 17, Loss: 1.179444
Epoch: 18, Loss: 1.111068
Epoch: 19, Loss: 1.049361
Epoch: 20, Loss: 0.990033
Epoch: 21, Loss: 0.935735
Epoch: 22, Loss: 0.884024
Epoch: 23, Loss: 0.836145
Epoch: 24, Loss: 0.791498
Epoch: 25, Loss: 0.748359
Epoch: 26, Loss: 0.708358
Epoch: 27, Loss: 0.669982
Epoch: 28, Loss: 0.635606
Epoch: 29, Loss: 0.603321
Epoch: 30, Loss: 0.572181


In [222]:
with torch.no_grad():
    def predict(model , question, threshold):
      numerical_question = text_to_indices(question, vocab)
      question = torch.tensor(numerical_question).unsqueeze(0)
      output, hidden = model(question)

      probs = F.softmax(output, dim=1)
      value , idx = torch.max(probs, dim=1)
      print(value,idx)

      if value < threshold:
        return "I don't know"
      else:
         print(list(vocab.keys())[idx])



In [231]:
predict(model, "What is the boiling point of water in Celsius ", 0.5)

tensor([0.9937], grad_fn=<MaxBackward0>) tensor([28])
100
